In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2

### 배치 정규화 클래스 정의

In [ ]:
class BatchNormalization(layers.BatchNormalization):
    def call(self, x, training=False):
        if not training:
            training = tf.constant(False)
        training = tf.logical_and(training, self.trainable)
        return super().call(x, training)

### 배치 정규화와 LeakyReLU 활성화 함수를 사용하는 합성곱 함수

In [ ]:
def convolutional(input_layer, filters, kernel_size, downsample=False, activate=True, bn=True):
    if downsample:
        input_layer = layers.ZeroPadding2D(((1,0), (1,0)))(input_layer)
        padding = 'valid'
        strides = 2
    else:
        strides = 1
        padding = 'same'

    kernel_init = tf.random_normal_initializer(stddev=0.01)
    conv = layers.Conv2D(filters=filters, kernel_size=kernel_size,
                         strides=strides, padding=padding,
                         use_bias=not bn,
                         kernel_initializer=kernel_init,
                         kernel_regularizer=l2(0.0005)
                        )(input_layer)
    if bn:
        conv = BatchNormalization()(conv)
    if activate:
        conv = layers.LeakyReLU(alpha=0.1)(conv)

    return conv

### Rediual 블록

In [ ]:
def residual_block(input_layer, filter_num1, filter_num2):
    short_cut = input_layer
    conv = convolutional(input_layer, filters=filter_num1, kernel_size=(1, 1))
    conv = convolutional(conv, filters=filter_num2, kernel_size=(3, 3))
    residual_output = short_cut + conv
    return residual_output

### Darknet-53

In [ ]:
def darknet53(input_data):
    input_data = convolutional(input_data, 32, (3, 3))
    input_data = convolutional(input_data, 64, (3, 3), downsample=True)

    for i in range(1):
        input_data = residual_block(input_data, 32, 64)

    input_data = convolutional(input_data, 128, (3, 3), downsample=True)

    for i in range(2):
        input_data = residual_block(input_data, 64, 128)

    input_data = convolutional(input_data, 256, (3, 3), downsample=True)

    for i in range(8):
        input_data = residual_block(input_data, 128, 256)

    route_1 = input_data
    input_data = convolutional(input_data, 512, (3, 3), downsample=True)

    for i in range(8):
        input_data = residual_block(input_data, 256, 512)

    route_2 = input_data
    input_data = convolutional(input_data, 1024, (3, 3), downsample=True)

    for i in range(4):
        input_data = residual_block(input_data, 512, 1024)

    return route_1, route_2, input_data

In [ ]:
from tensorflow.keras.layers import Input
input = Input(shape=(416, 416, 3))
backbone = darknet53(input)

In [ ]:
backbone

(<KerasTensor: shape=(None, 52, 52, 256) dtype=float32 (created by layer 'tf.__operators__.add_79')>,
 <KerasTensor: shape=(None, 26, 26, 512) dtype=float32 (created by layer 'tf.__operators__.add_87')>,
 <KerasTensor: shape=(None, 13, 13, 1024) dtype=float32 (created by layer 'tf.__operators__.add_91')>)

In [ ]:
from tensorflow.keras.models import Model

# 모델 정의
model = Model(inputs=input, outputs=backbone[2])
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 416, 416, 3)]        0         []                            
                                                                                                  
 conv2d_156 (Conv2D)         (None, 416, 416, 32)         864       ['input_4[0][0]']             
                                                                                                  
 batch_normalization_156 (B  (None, 416, 416, 32)         128       ['conv2d_156[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 leaky_re_lu_156 (LeakyReLU  (None, 416, 416, 32)         0         ['batch_normalization_15